# Novozyme Enzyme Stability Prediction

This notebook contains model training and evaluation to predict the thermal stability (as measured via melting point) of enzymes based on their amino acid sequence.

Competition details are available [here](https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/overview).

## Dataset Download

Retrieve the dataset from kaggle via the kaggle API

In [5]:
from pathlib import Path
from shlib import Cmd
import zipfile as 

In [11]:
# Path constants
DATA_DIR = Path("rawdata")
METADATA_DIR = Path("metadata")
LOG_DIR = Path("logs")
RESULT_DIR = Path("results")

# Kaggle constants
COMPETITION_NAME = "novozymes-enzyme-stability-prediction"


In [13]:
# Initialize project directories
for d in (DATA_DIR, METADATA_DIR, LOG_DIR, RESULT_DIR):
    d.mkdir(parents=True, exist_ok=True)

In [9]:
# Download and decompress competition data
download_competition_files = Cmd(["kaggle", "competitions", "download", "-c", 
    COMPETITION_NAME, "-p", DATA_DIR])
download_competition_files.run()


100%|██████████| 7.06M/7.06M [00:00<00:00, 43.8MB/s]


0

In [ ]:
dataset_download = Cmd(["kaggle", "datasets", "download", "-d", COMPETITION_NAME])